In [24]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [25]:
# Constants
SIGMA = 5.67e-8  # Stefan-Boltzmann constant (W/m²K⁴)
EMISSIVITY = 0.95

# Load your dataset
df = gpd.read_file("C:/Users/Dila Ozberkman/Desktop/AMS Research/Urban Shade/throwing_shade/code/results/output/e6a08fb8/climatebike_shade_results.geojson")

In [26]:
df.columns

Index(['TIMESTAMP', 'RECORD', 'GPSData', 'Air_Temp', 'humidity', 'Qs_in',
       'Qs_out', 'Qs_left', 'Qs_right', 'Qs_front', 'Qs_back', 'QL_in',
       'QL_out', 'QL_left', 'QL_right', 'QL_front', 'QL_back', 'TL_inout',
       'TL_leftright', 'TL_front', 'TL_back', 'wheelfreq', 'WindDir', 'WS_ms',
       'WSDiag', 'PTemp', 'batt_volt', 'GPSData_time', 'latitude', 'longitude',
       'building_shade', 'tree_shade', 'bldg_shadow_fraction',
       'tree_shadow_fraction', 'bldg_hrs_before_shadow_fraction',
       'tree_hrs_before_shadow_fraction', 'geometry'],
      dtype='object')

In [27]:
# List of columns to convert
numeric_cols = [
    "Air_Temp", "humidity", "PTemp", "WS_ms",
    "Qs_in", "Qs_out", "Qs_left", "Qs_right", "Qs_front", "Qs_back",
    "QL_in", "QL_out", "QL_left", "QL_right", "QL_front", "QL_back"
]

# Convert to float, handling errors gracefully
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)


In [28]:
# Convert sensor temperature to Kelvin
df["PTemp_K"] = df["PTemp"] + 273.15

# Correct longwave radiation for all directions
lw_columns = ["QL_in", "QL_out", "QL_left", "QL_right", "QL_front", "QL_back"]
for col in lw_columns:
    df[f"{col}_corrected"] = df[col] + SIGMA * df["PTemp_K"]**4

# Compute total absorbed radiation
df["Q_total"] = (
    df["Qs_in"] + df["Qs_out"] + df["Qs_left"] + df["Qs_right"] + df["Qs_front"] + df["Qs_back"] +
    df["QL_in_corrected"] + df["QL_out_corrected"] + df["QL_left_corrected"] +
    df["QL_right_corrected"] + df["QL_front_corrected"] + df["QL_back_corrected"]
)

# Compute MRT
df["MRT"] = ((df["Q_total"] / (SIGMA * EMISSIVITY)) ** 0.25) - 273.15  # Convert to Celsius

# Adjust for wind speed (Optional)
df["MRT_adj"] = df["MRT"] + 0.95 * np.sqrt(df["WS_ms"]) * (df["Air_Temp"] - df["MRT"])

In [14]:
df.columns

Index(['TIMESTAMP', 'RECORD', 'GPSData', 'Air_Temp', 'humidity', 'Qs_in',
       'Qs_out', 'Qs_left', 'Qs_right', 'Qs_front', 'Qs_back', 'QL_in',
       'QL_out', 'QL_left', 'QL_right', 'QL_front', 'QL_back', 'TL_inout',
       'TL_leftright', 'TL_front', 'TL_back', 'wheelfreq', 'WindDir', 'WS_ms',
       'WSDiag', 'PTemp', 'batt_volt', 'GPSData_time', 'latitude', 'longitude',
       'building_shade', 'tree_shade', 'bldg_shadow_fraction',
       'tree_shadow_fraction', 'bldg_hrs_before_shadow_fraction',
       'tree_hrs_before_shadow_fraction', 'geometry', 'PTemp_K',
       'QL_in_corrected', 'QL_out_corrected', 'QL_left_corrected',
       'QL_right_corrected', 'QL_front_corrected', 'QL_back_corrected',
       'Q_total', 'MRT', 'MRT_adj'],
      dtype='object')

In [16]:
df = df.drop(["QL_in", "QL_out", "QL_left", "QL_right", "QL_front", "QL_back"], axis=1)

In [22]:
df["Air_Temp"]

0        25.33
1        25.29
2        25.29
3        25.22
4        25.13
         ...  
33900    21.61
33901    21.62
33902    21.61
33903    21.56
33904    21.53
Name: Air_Temp, Length: 33905, dtype: float64

In [29]:
df

,TIMESTAMP,RECORD,GPSData,Air_Temp,humidity,Qs_in,Qs_out,Qs_left,Qs_right,Qs_front,...,PTemp_K,QL_in_corrected,QL_out_corrected,QL_left_corrected,QL_right_corrected,QL_front_corrected,QL_back_corrected,Q_total,MRT,MRT_adj
0,8/23/2023 12:57,56706,230823115735N5222666E00452996G004-00018E0000N0...,25.33,48.43,38.70,2.243,14.95,8.24,19.76,...,299.22,434.082188,443.512188,447.948188,451.979188,444.972188,443.002188,2766.079126,202.885656,9.089534
1,8/23/2023 12:57,56707,230823115736N5222666E00452996G004-00018E0000N0...,25.29,48.43,38.70,2.243,15.17,8.24,19.76,...,299.22,434.082188,443.512188,447.948188,452.094188,444.972188,445.412188,2767.774126,202.958566,-7.177236
2,8/23/2023 12:57,56708,230823115737N5222666E00452996G004-00018E0000N0...,25.29,48.46,38.70,2.243,15.17,8.24,19.76,...,299.22,434.082188,443.622188,447.948188,452.094188,444.972188,448.090188,2768.272126,202.979981,24.333051
3,8/23/2023 12:57,56709,230823115738N5222666E00452996G004-00018E0000N0...,25.22,48.45,38.92,2.243,15.17,8.24,19.76,...,299.22,434.082188,443.622188,447.948188,452.094188,444.972188,447.020188,2768.882126,203.006208,33.266927
4,8/23/2023 12:57,56710,230823115739N5222666E00452996G004-00018E0000N0...,25.13,48.38,38.92,2.243,15.17,8.24,19.76,...,299.22,434.082188,443.732188,447.948188,452.210188,444.972188,446.352188,2769.070126,203.014290,-34.775824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33900,9/12/2023 15:39,91000,230912143848N5222510E00454856G004+00015E0000N0...,21.61,86.00,50.61,1.122,8.79,31.90,22.61,...,298.67,435.799612,451.949612,446.552612,443.926612,445.096612,443.285612,2797.702674,204.240446,155.167540
33901,9/12/2023 15:39,91001,230912143848N5222510E00454856G004+00015E0000N0...,21.62,86.00,50.82,1.122,8.79,31.90,22.61,...,298.67,435.799612,451.949612,446.659612,444.157612,445.215612,443.149612,2798.443674,204.272054,152.216218
33902,9/12/2023 15:39,91002,230912143848N5222510E00454856G004+00015E0000N0...,21.61,85.90,50.82,1.346,8.79,32.12,22.61,...,298.67,435.799612,451.949612,446.659612,444.157612,445.215612,442.889612,2798.627674,204.279901,169.572620
33903,9/12/2023 15:39,91003,230912143848N5222510E00454856G004+00015E0000N0...,21.56,85.90,50.82,1.122,8.79,32.33,22.61,...,298.67,435.799612,451.949612,446.659612,444.157612,445.215612,442.749612,2798.683674,204.282289,121.033284
